<a href="https://colab.research.google.com/github/noobiantech/Coursera_Capstone/blob/main/CapstoneFridge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
Strategy for black lives (startegyforblacklives.org) wants to explore ways to reduce food waste across each borough. One action the organization plans on taking is strategically placing food banks to preserve food after food venues discard them. 

# Method 

Find all the food venues in a certain borough.

In each neighborhood list the top food venues 

Cluster neighborhoods by similarity between the top food spots in each area

The goal is to place food banks in an accessible place 


# Data

Look for chains and companies who often have wasted food and or 

find the most common types of food in each neighborhood and recommend what types of food each person would like from their respective food bank/ community fridge

Share food waste with neighborhoods that are in the same cluster

our goal is to create a food bank that is equipped to hold on to food that is similiar to eachother

clustering neighborhoods by the similiarity of their resturants we can house food waste from each neighborhood in a food bank close to the center of the cluster

use k mediods - pick points as the center so that we can utilize their resources ( electricity 



In [186]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset


Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 


For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.


In [187]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


#### Load and explore the data


Next, let's load the data.


In [188]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Let's take a quick look at the data.


In [189]:
#newyork_data

Notice how all the relevant data is in the _features_ key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.


In [190]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.


In [191]:
neighborhoods_data[0]

{'geometry': {'coordinates': [-73.84720052054902, 40.89470517661],
  'type': 'Point'},
 'geometry_name': 'geom',
 'id': 'nyu_2451_34572.1',
 'properties': {'annoangle': 0.0,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661],
  'borough': 'Bronx',
  'name': 'Wakefield',
  'stacked': 1},
 'type': 'Feature'}

#### Tranform the data into a _pandas_ dataframe


The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.


In [192]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.


In [193]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.


In [194]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.


In [195]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [196]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.


In [197]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.


However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.


In [198]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


Let's get the geographical coordinates of Manhattan.


In [199]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.


In [200]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [201]:
CLIENT_ID = 'HX3TUJWENCPPFYVCDNDVEEL4SOJ4ND5YVENJDMHIMW0JCK31' # your Foursquare ID
CLIENT_SECRET = '0JAQDTGQ2MDBEH1AUBO4UL2NZU3OFKKFQAPO5BZF35VZ3R3E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HX3TUJWENCPPFYVCDNDVEEL4SOJ4ND5YVENJDMHIMW0JCK31
CLIENT_SECRET:0JAQDTGQ2MDBEH1AUBO4UL2NZU3OFKKFQAPO5BZF35VZ3R3E


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [202]:
manhattan_data.loc[2, 'Neighborhood']

'Washington Heights'

Get the neighborhood's latitude and longitude values.


In [203]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964, -73.91065965862981.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [204]:
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore'

Double-click **here** for the solution.

<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 


Send the GET request and examine the resutls


In [205]:
params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v='20180323',
ll='40.7243,-74.0018',
 query='food',
limit=LIMIT
)

resp = requests.get(url=url, params=params).json()
#data = json.loads(resp.text)

results = resp
#results

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [206]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [207]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,SUGARFISH by sushi nozawa,Sushi Restaurant,40.725187,-74.003593
1,Chobani,Snack Place,40.725819,-74.000985
2,Boqueria,Tapas Restaurant,40.724800,-74.002220
3,San Carlo Osteria Piemonte,Italian Restaurant,40.725223,-74.002594
4,Blue Ribbon Brasserie,American Restaurant,40.725495,-74.003186


And how many venues were returned by Foursquare?


In [208]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in Manhattan


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [209]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.


In [210]:
# type your answer here
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


Double-click **here** for the solution.

<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 


#### Let's check the size of the resulting dataframe


In [211]:
print(manhattan_venues.shape)
manhattan_venues.head()

(2772, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
2,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
3,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
4,Marble Hill,40.876551,-73.91066,Subway,40.874667,-73.909586,Sandwich Place


Let's check how many venues were returned for each neighborhood


In [212]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,28,28,28,28,28,28
Carnegie Hill,69,69,69,69,69,69
Central Harlem,42,42,42,42,42,42
Chelsea,82,82,82,82,82,82
Chinatown,100,100,100,100,100,100
Civic Center,79,79,79,79,79,79
Clinton,100,100,100,100,100,100
East Harlem,54,54,54,54,54,54
East Village,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues


In [213]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 124 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [214]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Peking Duck Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Portuguese Restaurant,Poutine Place,Puerto Rican Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [215]:
manhattan_onehot.shape

(2772, 125)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [216]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Belgian Restaurant,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Chinese Restaurant,Creperie,Cuban Restaurant,Czech Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,German Restaurant,Gluten-free Restaurant,Greek Restaurant,Hawaiian Restaurant,Himalayan Restaurant,Hot Dog Joint,Hotpot Restaurant,Indian Restaurant,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jewish Restaurant,Kebab Restaurant,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Lebanese Restaurant,Mac & Cheese Joint,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Molecular Gastronomy Restaurant,Moroccan Restaurant,New American Restaurant,Noodle House,North Indian Restaurant,Paella Restaurant,Pakistani Restaurant,Peking Duck Restaurant,Persian Restaurant,Peruvian Restaurant,Pet Café,Pizza Place,Poke Place,Portuguese Restaurant,Poutine Place,Puerto Rican Restaurant,Ramen Restaurant,Restaurant,Russian Restaurant,Salad Place,Sandwich Place,Scandinavian Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Snack Place,Soba Restaurant,Soup Place,South American Restaurant,South Indian Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sri Lankan Restaurant,Steakhouse,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tex-Mex Restaurant,Thai Restaurant,Theme Restaurant,Tibetan Restaurant,Tonkatsu Restaurant,Turkish Restaurant,Udon Restaurant,Ukrainian Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wings Joint
0,Battery Park City,0.00,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.00,0.071429,0.000000,0.035714,0.00,0.035714,0.00,0.000000,0.071429,0.035714,0.00000,0.000000,0.000000,0.00,0.000000,0.00,0.071429,0.000000,0.000000,0.00,0.035714,0.00,0.000000,0.071429,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.035714,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.071429,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.035714,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.107143,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.107143,0.00,0.035714,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.035714,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000
1,Carnegie Hill,0.00,0.000000,0.014493,0.000000,0.014493,0.000000,0.000000,0.00,0.000000,0.014493,0.072464,0.00,0.000000,0.00,0.000000,0.014493,0.000000,0.00000,0.101449,0.000000,0.00,0.000000,0.00,0.028986,0.000000,0.000000,0.00,0.014493,0.00,0.043478,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.028986,0.000000,0.000000,0.000000,0.028986,0.043478,0.000000,0.000000,0.000000,0.00,0.014493,0.000000,0.00,0.014493,0.00,0.028986,0.00,0.000000,0.000000,0.086957,0.000000,0.028986,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000,0.00,0.000,0.014493,0.028986,0.000000,0.000000,0.000000,0.00,0.014493,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.101449,0.000000,0.00,0.00,0.000000,0.014493,0.028986

#### Let's confirm the new size


In [217]:
manhattan_grouped.shape

(40, 125)

#### Let's print each neighborhood along with the top 5 most common venues


In [218]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Pizza Place  0.11
1      Sandwich Place  0.11
2  Chinese Restaurant  0.07
3          Food Court  0.07
4          Donut Shop  0.07


----Carnegie Hill----
                venue  freq
0                Café  0.10
1         Pizza Place  0.10
2    Sushi Restaurant  0.10
3  Italian Restaurant  0.09
4              Bakery  0.07


----Central Harlem----
                             venue  freq
0              Fried Chicken Joint  0.10
1                    Deli / Bodega  0.10
2             Caribbean Restaurant  0.07
3               African Restaurant  0.07
4  Southern / Soul Food Restaurant  0.07


----Chelsea----
                 venue  freq
0    French Restaurant  0.09
1               Bakery  0.09
2                 Café  0.07
3  American Restaurant  0.06
4  Japanese Restaurant  0.05


----Chinatown----
                 venue  freq
0   Chinese Restaurant  0.17
1               Bakery  0.11
2   Dim Sum Restaurant  0.05
3    Hotpot Res

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [219]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [220]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Pizza Place,Sandwich Place,Italian Restaurant,Donut Shop,Food Court,BBQ Joint,Chinese Restaurant,Burger Joint,Mexican Restaurant,Deli / Bodega
1,Carnegie Hill,Sushi Restaurant,Pizza Place,Café,Italian Restaurant,Bakery,French Restaurant,Diner,Restaurant,Japanese Restaurant,Vietnamese Restaurant
2,Central Harlem,Fried Chicken Joint,Deli / Bodega,Southern / Soul Food Restaurant,African Restaurant,Caribbean Restaurant,Seafood Restaurant,French Restaurant,Sandwich Place,Restaurant,Chinese Restaurant
3,Chelsea,Bakery,French Restaurant,Café,American Restaurant,Japanese Restaurant,Pizza Place,Italian Restaurant,Seafood Restaurant,Mexican Restaurant,Sandwich Place
4,Chinatown,Chinese Restaurant,Bakery,Hotpot Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,American Restaurant,Pizza Place


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [221]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 2, 0, 1, 0, 2, 1, 1, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [222]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,2,Deli / Bodega,American Restaurant,Sandwich Place,Chinese Restaurant,Diner,Pizza Place,Seafood Restaurant,Snack Place,Steakhouse,Donut Shop
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Bakery,Hotpot Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,American Restaurant,Pizza Place
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Pizza Place,Deli / Bodega,Chinese Restaurant,Bakery,Café,Spanish Restaurant,Restaurant,Mexican Restaurant,Latin American Restaurant,Food Truck
3,Manhattan,Inwood,40.867684,-73.921210,1,Mexican Restaurant,Café,Spanish Restaurant,Pizza Place,Restaurant,Bakery,Thai Restaurant,Caribbean Restaurant,Deli / Bodega,Chinese Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Pizza Place,Mexican Restaurant,Deli / Bodega,Café,Latin American Restaurant,Donut Shop,Sandwich Place,Chinese Restaurant,Caribbean Restaurant,Sushi Restaurant


Finally, let's visualize the resulting clusters


In [223]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [224]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,American Restaurant,Pizza Place,French Restaurant,Bakery,Diner,Mexican Restaurant,Sushi Restaurant,Burger Joint,Sandwich Place
9,Yorkville,Italian Restaurant,Pizza Place,Deli / Bodega,Sushi Restaurant,Mexican Restaurant,Diner,Bakery,Japanese Restaurant,Chinese Restaurant,Donut Shop
10,Lenox Hill,Italian Restaurant,Sushi Restaurant,Pizza Place,Burger Joint,Deli / Bodega,Café,Chinese Restaurant,American Restaurant,Bakery,Diner
12,Upper West Side,Italian Restaurant,Indian Restaurant,Bakery,Mediterranean Restaurant,Restaurant,Pizza Place,Sushi Restaurant,Café,French Restaurant,Seafood Restaurant
13,Lincoln Square,Café,Food Truck,Italian Restaurant,American Restaurant,French Restaurant,Pizza Place,Mexican Restaurant,Deli / Bodega,Bakery,Chinese Restaurant
17,Chelsea,Bakery,French Restaurant,Café,American Restaurant,Japanese Restaurant,Pizza Place,Italian Restaurant,Seafood Restaurant,Mexican Restaurant,Sandwich Place
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Chinese Restaurant,Pizza Place,Café,French Restaurant,Sandwich Place,American Restaurant,Indian Restaurant,Mediterranean Restaurant
22,Little Italy,Chinese Restaurant,Italian Restaurant,Café,Bakery,Mediterranean Restaurant,Seafood Restaurant,Vietnamese Restaurant,Pizza Place,Hotpot Restaurant,French Restaurant
23,Soho,Italian Restaurant,Café,French Restaurant,Mediterranean Restaurant,Sandwich Place,Bakery,Seafood Restaurant,American Restaurant,Chinese Restaurant,Pizza Place
24,West Village,Italian Restaurant,American Restaurant,New American Restaurant,Café,French Restaurant,Pizza Place,Seafood Restaurant,Mexican Restaurant,Indian Restaurant,Chinese Restaurant


#### Cluster 2


In [225]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Chinese Restaurant,Bakery,Hotpot Restaurant,Dumpling Restaurant,Dim Sum Restaurant,Vietnamese Restaurant,Noodle House,Vegetarian / Vegan Restaurant,American Restaurant,Pizza Place
2,Washington Heights,Pizza Place,Deli / Bodega,Chinese Restaurant,Bakery,Café,Spanish Restaurant,Restaurant,Mexican Restaurant,Latin American Restaurant,Food Truck
3,Inwood,Mexican Restaurant,Café,Spanish Restaurant,Pizza Place,Restaurant,Bakery,Thai Restaurant,Caribbean Restaurant,Deli / Bodega,Chinese Restaurant
4,Hamilton Heights,Pizza Place,Mexican Restaurant,Deli / Bodega,Café,Latin American Restaurant,Donut Shop,Sandwich Place,Chinese Restaurant,Caribbean Restaurant,Sushi Restaurant
7,East Harlem,Mexican Restaurant,Pizza Place,Bakery,Deli / Bodega,Café,Latin American Restaurant,Thai Restaurant,Donut Shop,Sandwich Place,Fast Food Restaurant
19,East Village,Pizza Place,Mexican Restaurant,Vietnamese Restaurant,Korean Restaurant,Vegetarian / Vegan Restaurant,Chinese Restaurant,Italian Restaurant,Ramen Restaurant,Japanese Restaurant,Seafood Restaurant
20,Lower East Side,Bakery,Pizza Place,Chinese Restaurant,Deli / Bodega,Café,Mexican Restaurant,Sandwich Place,Japanese Restaurant,Ramen Restaurant,Diner
25,Manhattan Valley,Pizza Place,Mexican Restaurant,Indian Restaurant,Thai Restaurant,Café,Latin American Restaurant,French Restaurant,Noodle House,Burger Joint,Fried Chicken Joint


#### Cluster 3


In [226]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Deli / Bodega,American Restaurant,Sandwich Place,Chinese Restaurant,Diner,Pizza Place,Seafood Restaurant,Snack Place,Steakhouse,Donut Shop
5,Manhattanville,Deli / Bodega,Chinese Restaurant,Sushi Restaurant,Mexican Restaurant,Seafood Restaurant,Café,Latin American Restaurant,Sandwich Place,Italian Restaurant,Diner
6,Central Harlem,Fried Chicken Joint,Deli / Bodega,Southern / Soul Food Restaurant,African Restaurant,Caribbean Restaurant,Seafood Restaurant,French Restaurant,Sandwich Place,Restaurant,Chinese Restaurant
11,Roosevelt Island,Restaurant,Deli / Bodega,Greek Restaurant,Food,Japanese Restaurant,Kosher Restaurant,Sandwich Place,Café,Pizza Place,Chinese Restaurant
14,Clinton,Italian Restaurant,Deli / Bodega,American Restaurant,Restaurant,Sandwich Place,Chinese Restaurant,Thai Restaurant,Pizza Place,Mexican Restaurant,Café
15,Midtown,American Restaurant,Sandwich Place,Burger Joint,Japanese Restaurant,Steakhouse,Sushi Restaurant,Cuban Restaurant,French Restaurant,Bakery,Mediterranean Restaurant
16,Murray Hill,Sandwich Place,American Restaurant,Japanese Restaurant,Mediterranean Restaurant,Salad Place,Pizza Place,Deli / Bodega,Burger Joint,Restaurant,Taco Place
21,Tribeca,American Restaurant,Italian Restaurant,Café,Deli / Bodega,Sandwich Place,Greek Restaurant,Bakery,Burger Joint,French Restaurant,Food Truck
26,Morningside Heights,Deli / Bodega,Café,Food Truck,Pizza Place,American Restaurant,Burger Joint,Chinese Restaurant,New American Restaurant,Sandwich Place,Mexican Restaurant
27,Gramercy,Bagel Shop,Italian Restaurant,Thai Restaurant,American Restaurant,Pizza Place,Mexican Restaurant,Sandwich Place,Diner,Deli / Bodega,Taco Place


#### Cluster 4


In [227]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,German Restaurant,Deli / Bodega,Bistro,Gastropub,Egyptian Restaurant,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant


#### Cluster 5


In [228]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [ ]:
def getCenterLoc(cluster):
  cent_lat = cluster['latitude'].mean()
  cent_long = cluster['longtitude'].mean()
  return (cent_latitude )